# Определение пола человека по его имени
Работу выполнили студентки 2 курса Высшей школы ИТИС Грибанова Анастасия и Шафеева Дина, группа 11-803.

**Цель проекта:** научиться по имени, фамилии и отчеству человека определять его пол. 

Для достижения этой цели берутся данные 62609 человек и на их основе строятся различные модели.
Для работы берутся данные русскоязычного населения, из-за наличия у тех отчества, которое будет являться дополнительным параметром для анализа.

**Поля:** полями данных являются полные имена, а именно: фамилия, имя, отчество человека. Последнее поле в таблице - пол.

**Источник:** https://github.com/Rai220/MlSexDetector/blob/master/dataMini.csv

In [1]:
import chardet
import pandas as pd

with open('C:/Users/Nastya/dataMini.csv', 'rb') as f:
    result = chardet.detect(f.readline()) 


data = pd.read_csv('C:/Users/Nastya/dataMini.csv', encoding=result['encoding'])
data.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М
